In [1]:
import geopandas as gpd
import requests
import pandas as pd
from shapely.geometry import shape


In [10]:
stations_url = "https://api.energyandcleanair.org/stations?country=GB,US,TR,PH,IN,TH&format=geojson"
stations_respons = requests.get(stations_url)
stations_data = stations_respons.json()

stations_gdf = gpd.GeoDataFrame.from_features(stations_data['features'])
stations_gdf = stations_gdf.set_crs("EPSG:4326")


In [11]:
#Fetching data
countries_url = "https://datahub.io/core/geo-countries/r/countries.geojson"
countries_respons = requests.get(countries_url)
countries_data = countries_response.json()

# GeoDataFrame
countries_gdf = gpd.GeoDataFrame.from_features(countries_data['features'])
countries_gdf = countries_gdf.set_crs("EPSG:4326")


In [14]:
#test in order to know the dataset variable for country(?) target
print(countries_data['features'][0])

{'type': 'Feature', 'properties': {'ADMIN': 'Aruba', 'ISO_A3': 'ABW', 'ISO_A2': 'AW'}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-69.99693762899992, 12.577582098000036], [-69.93639075399994, 12.53172435100005], [-69.92467200399994, 12.519232489000046], [-69.91576087099992, 12.497015692000076], [-69.88019771999984, 12.453558661000045], [-69.87682044199994, 12.427394924000097], [-69.88809160099993, 12.417669989000046], [-69.90880286399994, 12.417792059000107], [-69.93053137899989, 12.425970770000035], [-69.94513912699992, 12.44037506700009], [-69.92467200399994, 12.44037506700009], [-69.92467200399994, 12.447211005000014], [-69.95856686099992, 12.463202216000099], [-70.02765865799992, 12.522935289000088], [-70.04808508999989, 12.53115469000008], [-70.05809485599988, 12.537176825000088], [-70.06240800699987, 12.546820380000057], [-70.06037350199995, 12.556952216000113], [-70.0510961579999, 12.574042059000064], [-70.04873613199993, 12.583726304000024], [-70.05264238199993, 12

In [15]:
# List countries
target_countries = ['United States', 'United Kingdom', 'Turkey', 'Thailand', 'Philippines', 'India']

# Filter the countries
countries_gdf = countries_gdf[countries_gdf['ADMIN'].isin(target_countries)]


In [16]:
# Placeholder results
results = []

# Iteration
for _, country in countries_gdf.iterrows():
    country_name = country['ADMIN']
    country_geometry = country['geometry']
    
    # Find the stations 
    stations_in_country = stations_gdf[stations_gdf.geometry.within(country_geometry)]
    
    # Calculate the number of stations in listed countries
    num_stations = len(stations_in_country)
    
    # Calculate the area (using metric)
    country_area = country_geometry.area / 10**6  
    
    # Calculate the density 
    density = (num_stations / country_area) * 1000
    
    # Append it
    results.append({
        'Country Name': country_name,
        'Number of PM10 Stations': num_stations,
        'Area (sq. km)': country_area,
        'Density of PM10 Stations per 1,000 sq. km': density
    })


In [17]:

results_df = pd.DataFrame(results)


results_df_sorted = results_df.sort_values(by='Density of PM10 Stations per 1,000 sq. km', ascending=False)


results_df_sorted


,Country Name,Number of PM10 Stations,Area (sq. km),"Density of PM10 Stations per 1,000 sq. km"
0,United Kingdom,422,0.000034,1.257984e+10
3,Thailand,340,0.000043,7.847862e+09
4,Turkey,369,0.000081,4.543806e+09
1,India,552,0.000279,1.981063e+09
2,Philippines,19,0.000024,7.796080e+08
